In [2]:
import os
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import nltk

# Prepare Corpus

In [4]:
text = open("large_text.txt").read()

nltk.download('punkt_tab')
CORPUS = nltk.sent_tokenize(text)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lilyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def build_tokenizer(texts, oov_token="<OOV>", lower=True, num_words=None):
    tok = Tokenizer(oov_token=oov_token, lower=lower, num_words=num_words)
    tok.fit_on_texts(texts)
    return tok

def make_ngram_dataset(tokenizer, texts, min_len=2):
    """
    Builds n-gram sequences for next-word prediction.
    Each sequence[i] is tokens[0:i] -> predict tokens[i]
    """
    sequences = []
    for line in texts:
        tokens = tokenizer.texts_to_sequences([line])[0]
        # create incremental n-grams: [w1,w2]->w3, [w1,w2,w3]->w4, ...
        for i in range(min_len, len(tokens) + 1):
            sequences.append(tokens[:i])
    if not sequences:
        raise ValueError("No sequences constructed; check your corpus/tokenizer.")

    max_len = max(len(s) for s in sequences)
    sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
    X, y = sequences[:, :-1], sequences[:, -1]
    vocab_size = min(tokenizer.num_words or len(tokenizer.word_index) + 1,
                     len(tokenizer.word_index) + 1)
    return X, y, max_len, vocab_size